In [1]:
""" combine query predictions and output as a result json file """
import os
from utils import execute_all

path1 = "./RESULTS/baselinev3-flan-t5-large_fold8/predictions_0.json"
path2 = "./RESULTS/baselinev3-flan-t5-large_fold8/predictions_1.json"

# combine two json files
import json
with open(path1) as f:
    data1 = json.load(f)
with open(path2) as f:
    data2 = json.load(f)

data1.update(data2)

DB_PATH = os.path.join('data', 'mimic_iv', 'mimic_iv.sqlite')
predictions = execute_all(data1, db_path=DB_PATH, tag='pred')

with open("./RESULTS/baselinev3-flan-t5-large_fold8/predictions.json", "w") as f:
    json.dump(predictions, f, indent=4)

### Hard Vote Ensemble

In [53]:
import json

PATHS = [  
  "./RESULTS/baselinev3-flan-t5-large_fold0/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold1/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold2/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold3/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold4/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold5/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold6/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold7/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold8/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold9/predictions.json",
]

In [54]:
import pandas as pd

df = pd.DataFrame(columns=["id"])
for idx, json_path in enumerate(PATHS):
  column_name = f"answer_{idx}"
  df[column_name] = None
  with open(json_path, "r") as f:
    data = json.load(f)
    if idx == 0: # initialize column
      df["id"] = list(data.keys())
    for key in data: # append data
      df.loc[df["id"] == key, column_name] = data[key]

df.head(10)

,id,answer_0,answer_1,answer_2,answer_3,answer_4,answer_5,answer_6,answer_7,answer_8,answer_9
0,282f008dd8dfb8f4a1dd6999,[['enterococcus sp.']],[['enterococcus sp.']],[['enterococcus sp.']],[['enterococcus sp.']],[['enterococcus sp.']],[['enterococcus sp.']],[['enterococcus sp.']],[['enterococcus sp.']],[['enterococcus sp.']],[['enterococcus sp.']]
1,7f07c59357750ac2b84e5221,"[['blood culture'], ['bronchoalveolar lavage'], ['fluid received in blood culture bottles'], ['peritoneal fluid'], ['rapid respiratory viral screen & culture'], ['sputum'], ['stool']]","[['blood culture'], ['bronchoalveolar lavage'], ['fluid received in blood culture bottles'], ['peritoneal fluid'], ['rapid respiratory viral screen & culture'], ['sputum'], ['stool']]","[['blood culture'], ['bronchoalveolar lavage'], ['fluid received in blood culture bottles'], ['peritoneal fluid'], ['rapid respiratory viral screen & culture'], ['sputum'], ['stool']]","[['blood culture'], ['bronchoalveolar lavage'], ['fluid received in blood culture bottles'], ['peritoneal fluid'], ['rapid respiratory viral screen & culture'], ['sputum'], ['stool']]","[['blood culture'], ['bronchoalveolar lavage'], ['fluid received in blood culture bottles'], ['peritoneal fluid'], ['rapid respiratory viral screen & culture'], ['sputum'], ['stool']]","[['blood culture'], ['bronchoalveolar lavage'], ['fluid received in blood culture bottles'], ['peritoneal fluid'], ['rapid respiratory viral screen & culture'], ['sputum'], ['stool']]","[['blood culture'], ['bronchoalveolar lavage'], ['fluid received in blood culture bottles'], ['peritoneal fluid'], ['rapid respiratory viral screen & culture'], ['sputum'], ['stool']]","[['blood culture'], ['bronchoalveolar lavage'], ['fluid received in blood culture bottles'], ['peritoneal fluid'], ['rapid respiratory viral screen & culture'], ['sputum'], ['stool']]","[['blood culture'], ['bronchoalveolar lavage'], ['fluid received in blood culture bottles'], ['peritoneal fluid'], ['rapid respiratory viral screen & culture'], ['sputum'], ['stool']]","[['blood culture'], ['bronchoalveolar lavage'], ['fluid received in blood culture bottles'], ['peritoneal fluid'], ['rapid respiratory viral screen & culture'], ['sputum'], ['stool']]"
2,f430a02bd152c617a86cd2c3,[],[['8.2']],"[['6.72'], ['7.53'], ['8.72'], ['9.12']]",[['4721.95']],[['None']],"[['10.06'], ['10.07'], ['10.1'], ['10.2'], ['10.29'], ['10.3'], ['10.34'], ['10.37'], ['10.38'], ['10.44'], ['10.59'], ['10.65'], ['10.72'], ['10.76'], ['10.78'], ['10.89'], ['10.9'], ['10.98'], ['11.0'], ['11.05'], ['11.16'], ['11.21'], ['11.38'], ['11.45'], ['11.49'], ['11.54'], ['11.55'], ['11.59'], ['11.61'], ['11.63'], ['11.64'], ['11.76'], ['11.77'], ['11.82'], ['11.89'], ['11.91'], ['11.93'], ['11.94'], ['12.12'], ['12.16'], ['12.17'], ['12.38'], ['12.44'], ['12.5'], ['12.51'], ['12.55'], ['12.57'], ['12.68'], ['12.72'], ['12.75'], ['12.78'], ['12.81'], ['12.87'], ['12.9'], ['13.06'], ['13.2'], ['13.21'], ['13.22'], ['13.27'], ['13.31'], ['13.43'], ['13.63'], ['13.69'], ['13.86'], ['5.86'], ['6.71'], ['6.72'], ['6.79'], ['7.01'], ['7.03'], ['7.07'], ['7.18'], ['7.34'], ['7.35'], ['7.37'], ['7.53'], ['7.54'], ['7.62'], ['7.79'], ['8.08'], ['8.4'], ['8.42'], ['8.58'], ['8.6'], ['8.72'], ['8.78'], ['8.84'], ['8.95'], ['8.96'], ['8.98'], ['9.12'], ['9.14'], ['9.22'], ['9.3'], ['9.32'], ['9.34'], ['9.37'], ['9.41'], ['9.49'], ['9.5']]",[],error_pred,[],[]
3,dfc058cc8e8d30a29c0177b9,[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']]
4,647bc08d18cac6dbda791c42,[['2100-10-30 16:00:00']],[['2100-10-30 16:00:00']],[['2100-10-30 16:00:00']],[['2100-10-30 16:00:00']],[['2100-10-30 16:00:00']],[['2100-10-30 16:00:00']],[['2100-10-30 16:00:00']],[['2100-10-30 16:00:00']],[['2100-10-30 16:00:00']],[['2100-10-30 16:00:00']]
5,5436413908d5e902536630e8,[['urine']],[['urine']],[['urine']],[['urine']],[['urine']],[['urine']],[['urine']],[['urine']],[['urine']],[['urine']]
6,4f489c24ce6a5dfb

In [55]:
# find rows with different answers
diff_row_idxs = []
for idx, row in df.iterrows():
  columns = list(row.keys())
  answer_columns = [col for col in columns if "answer" in col]
  answers = [row[col] for col in answer_columns]
  if len(set(answers)) > 1:
    diff_row_idxs.append(idx)

df_diff = df.loc[diff_row_idxs]
print(f"Number of different rows: {len(df_diff)}")
df_diff.head(10)

Number of different rows: 415


,id,answer_0,answer_1,answer_2,answer_3,answer_4,answer_5,answer_6,answer_7,answer_8,answer_9
2,f430a02bd152c617a86cd2c3,[],[['8.2']],"[['6.72'], ['7.53'], ['8.72'], ['9.12']]",[['4721.95']],[['None']],"[['10.06'], ['10.07'], ['10.1'], ['10.2'], ['10.29'], ['10.3'], ['10.34'], ['10.37'], ['10.38'], ['10.44'], ['10.59'], ['10.65'], ['10.72'], ['10.76'], ['10.78'], ['10.89'], ['10.9'], ['10.98'], ['11.0'], ['11.05'], ['11.16'], ['11.21'], ['11.38'], ['11.45'], ['11.49'], ['11.54'], ['11.55'], ['11.59'], ['11.61'], ['11.63'], ['11.64'], ['11.76'], ['11.77'], ['11.82'], ['11.89'], ['11.91'], ['11.93'], ['11.94'], ['12.12'], ['12.16'], ['12.17'], ['12.38'], ['12.44'], ['12.5'], ['12.51'], ['12.55'], ['12.57'], ['12.68'], ['12.72'], ['12.75'], ['12.78'], ['12.81'], ['12.87'], ['12.9'], ['13.06'], ['13.2'], ['13.21'], ['13.22'], ['13.27'], ['13.31'], ['13.43'], ['13.63'], ['13.69'], ['13.86'], ['5.86'], ['6.71'], ['6.72'], ['6.79'], ['7.01'], ['7.03'], ['7.07'], ['7.18'], ['7.34'], ['7.35'], ['7.37'], ['7.53'], ['7.54'], ['7.62'], ['7.79'], ['8.08'], ['8.4'], ['8.42'], ['8.58'], ['8.6'], ['8.72'], ['8.78'], ['8.84'], ['8.95'], ['8.96'], ['8.98'], ['9.12'], ['9.14'], ['9.22'], ['9.3'], ['9.32'], ['9.34'], ['9.37'], ['9.41'], ['9.49'], ['9.5']]",[],error_pred,[],[]
6,4f489c24ce6a5dfb8485d4f0,null,error_pred,null,error_pred,[['10026406.0']],error_pred,null,[['1.0']],error_pred,[]
8,289be48fc696c588e5a69f9c,error_pred,[['1.0']],[['1.0']],error_pred,[['0.0']],[['0.0']],[['1.0']],[],[['0.0']],error_pred
9,9e26fb10ef1cbf36ec4e516b,error_pred,null,[['0.0']],error_pred,null,null,null,null,error_pred,null
10,c59a0d9f2769a6133e2d4d28,error_pred,null,[],null,null,null,null,null,null,null
11,fa7797af0bc8368e7b76a129,[['None']],[['None']],[['None']],[['-6.0']],[['-6.0']],[['None']],[['-6.0']],[['None']],[['None']],[['None']]
13,d3f0d98539d2e9134a435816,[['0.0']],[['597.983']],[['597.983']],error_pred,[['597.983']],[],error_pred,[['597.983']],[['597.983']],error_pred
20,95259eb7cea07ea7f2a5cf48,error_pred,[['1.0']],error_pred,error_pred,[['99.0']],[['1.0']],error_pred,[['1.0']],[['1.0']],[['1.0']]
21,104c93f89e19274578bafb8d,[['2100-06-05 00:00:00']],[['2100-06-05 00:00:00']],[['2100-06-05 00:00:00']],[['1.0']],[['2100-06-05 00:00:00']],[['1.0']],[['2100-06-05 00:00:00']],[['2100-06-05 00:00:00']],[['2100-06-05 00:00:00']],[['2100-06-11 13:43:00']]
22,e0e182b93545a2f808a87836,null,error_pred,error_pred,[['None']],error_pred,null,null,error_pred,null,error_pred


In [5]:
# iterrate over rows and check error_pred
error_pred_idxs = []
for idx, row in df.iterrows():
  columns = list(row.keys())
  answer_columns = [col for col in columns if "answer" in col]
  answers = [row[col] for col in answer_columns]
  if "error_pred" in answers:
    error_pred_idxs.append(idx)

df_error_pred = df.loc[error_pred_idxs]
print(f"Number of error_pred rows: {len(df_error_pred)}")
df_error_pred.head(10)

Number of error_pred rows: 321


,id,answer_0,answer_1,answer_2,answer_3,answer_4,answer_5,answer_6,answer_7,answer_8,answer_9
2,f430a02bd152c617a86cd2c3,[],[['8.2']],"[['6.72'], ['7.53'], ['8.72'], ['9.12']]",[['4721.95']],[['None']],"[['10.06'], ['10.07'], ['10.1'], ['10.2'], ['1...",[],error_pred,[],[]
6,4f489c24ce6a5dfb8485d4f0,null,error_pred,null,error_pred,[['10026406.0']],error_pred,null,[['1.0']],error_pred,[]
8,289be48fc696c588e5a69f9c,error_pred,[['1.0']],[['1.0']],error_pred,[['0.0']],[['0.0']],[['1.0']],[],[['0.0']],error_pred
9,9e26fb10ef1cbf36ec4e516b,error_pred,null,[['0.0']],error_pred,null,null,null,null,error_pred,null
10,c59a0d9f2769a6133e2d4d28,error_pred,null,[],null,null,null,null,null,null,null
13,d3f0d98539d2e9134a435816,[['0.0']],[['597.983']],[['597.983']],error_pred,[['597.983']],[],error_pred,[['597.983']],[['597.983']],error_pred
20,95259eb7cea07ea7f2a5cf48,error_pred,[['1.0']],error_pred,error_pred,[['99.0']],[['1.0']],error_pred,[['1.0']],[['1.0']],[['1.0']]
22,e0e182b93545a2f808a87836,null,error_pred,error_pred,[['None']],error_pred,null,null,error_pred,null,error_pred
23,271fe9a047e24d8cef13cf91,error_pred,error_pred,[],error_pred,null,[],error_pred,[],[],error_pred
24,297a1435453b2870054eb81d,null,error_pred,error_pred,[['None']],error_pred,null,null,null,null,error_pred


In [6]:
# check for rows that either have error_pred or different answers
error_pred_diff_idxs = list(set(diff_row_idxs + error_pred_idxs))
df_error_pred_diff = df.loc[error_pred_diff_idxs]
print(f"Number of error_pred or different rows: {len(df_error_pred_diff)}")
df_error_pred_diff.head(10)

Number of error_pred or different rows: 419


,id,answer_0,answer_1,answer_2,answer_3,answer_4,answer_5,answer_6,answer_7,answer_8,answer_9
2,f430a02bd152c617a86cd2c3,[],[['8.2']],"[['6.72'], ['7.53'], ['8.72'], ['9.12']]",[['4721.95']],[['None']],"[['10.06'], ['10.07'], ['10.1'], ['10.2'], ['1...",[],error_pred,[],[]
6,4f489c24ce6a5dfb8485d4f0,null,error_pred,null,error_pred,[['10026406.0']],error_pred,null,[['1.0']],error_pred,[]
8,289be48fc696c588e5a69f9c,error_pred,[['1.0']],[['1.0']],error_pred,[['0.0']],[['0.0']],[['1.0']],[],[['0.0']],error_pred
9,9e26fb10ef1cbf36ec4e516b,error_pred,null,[['0.0']],error_pred,null,null,null,null,error_pred,null
10,c59a0d9f2769a6133e2d4d28,error_pred,null,[],null,null,null,null,null,null,null
11,fa7797af0bc8368e7b76a129,[['None']],[['None']],[['None']],[['-6.0']],[['-6.0']],[['None']],[['-6.0']],[['None']],[['None']],[['None']]
13,d3f0d98539d2e9134a435816,[['0.0']],[['597.983']],[['597.983']],error_pred,[['597.983']],[],error_pred,[['597.983']],[['597.983']],error_pred
20,95259eb7cea07ea7f2a5cf48,error_pred,[['1.0']],error_pred,error_pred,[['99.0']],[['1.0']],error_pred,[['1.0']],[['1.0']],[['1.0']]
21,104c93f89e19274578bafb8d,[['2100-06-05 00:00:00']],[['2100-06-05 00:00:00']],[['2100-06-05 00:00:00']],[['1.0']],[['2100-06-05 00:00:00']],[['1.0']],[['2100-06-05 00:00:00']],[['2100-06-05 00:00:00']],[['2100-06-05 00:00:00']],[['2100-06-11 13:43:00']]
22,e0e182b93545a2f808a87836,null,error_pred,error_pred,[['None']],error_pred,null,null,error_pred,null,error_pred


In [8]:
df_error_pred_diff.tail(10)

,id,answer_0,answer_1,answer_2,answer_3,answer_4,answer_5,answer_6,answer_7,answer_8,answer_9
1135,0d6e154c99ea7aa10dc0ac5c,[['50.0']],[['50.0']],[['50.0']],[['50.0']],[['50.0']],[['50.0']],[['None']],[['50.0']],[['50.0']],[['50.0']]
1137,207422a6435e8494054f044b,[['0.0']],error_pred,error_pred,[['0.0']],[['0.0']],[['0.0']],error_pred,[['0.0']],[['0.0']],[['0.0']]
1139,7ed6ef4b97a89c40124bd85f,error_pred,[['0.0']],[['0.0']],error_pred,error_pred,[['0.0']],[['0.0']],error_pred,[['0.0']],[['0.0']]
1140,ec1ed057dd7ec2a55f903e6e,[['0.0']],[['0.0']],error_pred,[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']],[['0.0']]
1144,debd4fb35717b39a57ee2bec,[['7.18']],[],[['7.18']],[['7.18']],[],[['7.18']],[],[],[['7.18']],[['7.18']]
1153,6985b7163e6c8b9db7102745,[['0.0']],error_pred,error_pred,[['9.0']],[['9.0']],[['9.0']],[['9.0']],[['0.0']],[['0.0']],[['0.0']]
1158,b79b244dc972c7cd9869e514,null,null,null,[],null,null,null,null,null,null
1159,9df13449259f96705f0f7f56,error_pred,error_pred,error_pred,[['None']],error_pred,[['None']],error_pred,error_pred,error_pred,error_pred
1161,8101d176c45f4e285b9e9540,error_pred,[['1.0']],[['0.0']],error_pred,error_pred,[['1.0']],error_pred,[['1.0']],[['1.0']],[['1.0']]
1164,e2740d72774b525d272c8e62,error_pred,error_pred,error_pred,[],[['0.0']],error_pred,error_pred,null,error_pred,[]


In [7]:
from collections import Counter

# write df as new json file
ensembled_dict = {}
for idx, row in df.iterrows():
  columns = list(row.keys())
  answer_columns = [col for col in columns if "answer" in col]
  answers = [row[col] for col in answer_columns]
  if "error_pred" in answers: # pass the difficult question
    ensembled_dict[row["id"]] = "null"
    continue
  else:
    # answer only when the frequency is more than half of the all answers
    answer_counter = Counter(answers) # calculate answers and its frequency
    max_freq = max(answer_counter.values())
    max_freq_answer = [key for key, value in answer_counter.items() if value == max_freq]
    if max_freq > len(answers) / 2:
      ensembled_dict[row["id"]] = max_freq_answer[0]
    else:
      ensembled_dict[row["id"]] = "null"

In [9]:
import json

%cd ./RESULTS/ENSEMBLE
path = "./test_ensemble_v4_ten.json"
zip_path = path.replace(".json", ".zip")
with open(path, "w") as f:
    json.dump(ensembled_dict, f, indent=4)

!zip $zip_path $path

/ext_hdd/yjahn/joint-learning/RESULTS/ENSEMBLE
  adding: test_ensemble_v4_ten.json (deflated 73%)


/home/yjahn/miniconda3/envs/ehr/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### Use Validation set to EDA

In [4]:
import json

PATHS = [  
  "./RESULTS/baselinev3-flan-t5-large_fold0/predictions_valid_0.json",
  "./RESULTS/baselinev3-flan-t5-large_fold0/predictions_valid_1.json",
  "./RESULTS/baselinev3-flan-t5-large_fold1/predictions_valid_0.json",
  "./RESULTS/baselinev3-flan-t5-large_fold1/predictions_valid_1.json",
  "./RESULTS/baselinev3-flan-t5-large_fold2/predictions_valid_0.json",
  "./RESULTS/baselinev3-flan-t5-large_fold2/predictions_valid_1.json",
  "./RESULTS/baselinev3-flan-t5-large_fold3/predictions_valid_0.json",
  "./RESULTS/baselinev3-flan-t5-large_fold3/predictions_valid_1.json",
  "./RESULTS/baselinev3-flan-t5-large_fold4/predictions_valid_0.json",
  "./RESULTS/baselinev3-flan-t5-large_fold4/predictions_valid_1.json",
  "./RESULTS/baselinev3-flan-t5-large_fold5/predictions_valid_0.json",
  "./RESULTS/baselinev3-flan-t5-large_fold5/predictions_valid_1.json",
  "./RESULTS/baselinev3-flan-t5-large_fold6/predictions_valid_0.json",
  "./RESULTS/baselinev3-flan-t5-large_fold6/predictions_valid_1.json",
  "./RESULTS/baselinev3-flan-t5-large_fold7/predictions_valid_0.json",
  "./RESULTS/baselinev3-flan-t5-large_fold7/predictions_valid_1.json",
  "./RESULTS/baselinev3-flan-t5-large_fold8/predictions_valid_0.json",
  "./RESULTS/baselinev3-flan-t5-large_fold8/predictions_valid_1.json",
  "./RESULTS/baselinev3-flan-t5-large_fold9/predictions_valid_0.json",
  "./RESULTS/baselinev3-flan-t5-large_fold9/predictions_valid_1.json",
]

In [5]:
# read json and gather 
import json

dict = {}

for path in PATHS:
  with open(path, "r") as f:
    data = json.load(f)
    dict.update(data)

# save dict as json
import json

path = "./train_pred_results.json"
with open(path, "w") as f:
    json.dump(dict, f, indent=4)

In [38]:
from scoring_program.postprocessing import post_process_sql
# compare with label
label_path = "/ext_hdd/yjahn/joint-learning/data/mimic_iv/train/label.json"
with open(label_path, "r") as f:
    label = json.load(f)

pairs = []
for id in label:
  if id not in dict:
    print(f"ID {id} is not in dict")
  pairs.append(
     (
        id,
        post_process_sql(dict[id]),
        post_process_sql(label[id])
      )
  )

In [39]:
pairs[0]

('3b9849548e56c59f768d5447',
 "SELECT MIN(chartevents.valuenum) FROM chartevents WHERE chartevents.stay_id IN ( SELECT icustays.stay_id FROM icustays WHERE icustays.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10021118 ) AND icustays.outtime IS NOT NULL ORDER BY icustays.intime ASC LIMIT 1 ) AND chartevents.itemid IN ( SELECT d_items.itemid FROM d_items WHERE d_items.label ='respiratory rate' AND d_items.linksto = 'chartevents' )",
 "SELECT MIN(chartevents.valuenum) FROM chartevents WHERE chartevents.stay_id IN ( SELECT icustays.stay_id FROM icustays WHERE icustays.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10021118 ) AND icustays.outtime IS NOT NULL ORDER BY icustays.intime ASC LIMIT 1 ) AND chartevents.itemid IN ( SELECT d_items.itemid FROM d_items WHERE d_items.label = 'respiratory rate' AND d_items.linksto = 'chartevents' )")

In [40]:
# make pairs into dataframe
import pandas as pd

df = pd.DataFrame(pairs, columns=["id", "pred", "label"])
df["pred"] = df["pred"].apply(lambda x: x.replace("='", "= '"))
df.head(10)

,id,pred,label
0,3b9849548e56c59f768d5447,SELECT MIN(chartevents.valuenum) FROM chartevents WHERE chartevents.stay_id IN ( SELECT icustays.stay_id FROM icustays WHERE icustays.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10021118 ) AND icustays.outtime IS NOT NULL ORDER BY icustays.intime ASC LIMIT 1 ) AND chartevents.itemid IN ( SELECT d_items.itemid FROM d_items WHERE d_items.label = 'respiratory rate' AND d_items.linksto = 'chartevents' ),SELECT MIN(chartevents.valuenum) FROM chartevents WHERE chartevents.stay_id IN ( SELECT icustays.stay_id FROM icustays WHERE icustays.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10021118 ) AND icustays.outtime IS NOT NULL ORDER BY icustays.intime ASC LIMIT 1 ) AND chartevents.itemid IN ( SELECT d_items.itemid FROM d_items WHERE d_items.label = 'respiratory rate' AND d_items.linksto = 'chartevents' )
1,3f720ea93f94700efe7fe7ee,"SELECT DISTINCT T1.C1 FROM ( SELECT labevents.valuenum, PERCENT_RANK() OVER ( ORDER BY labevents.valuenum ) AS C1 FROM labevents WHERE labevents.itemid IN ( SELECT d_labitems.itemid FROM d_labitems WHERE d_labitems.label = 'red blood cells' ) AND labevents.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.age = ( SELECT admissions.age FROM admissions WHERE admissions.subject_id = 10026406 AND admissions.dischtime IS NOT NULL ORDER BY admissions.admittime ASC LIMIT 1 ) ) ) AS T1 WHERE T1.valuenum = 4.46","SELECT DISTINCT T1.C1 FROM ( SELECT labevents.valuenum, PERCENT_RANK() OVER ( ORDER BY labevents.valuenum ) AS C1 FROM labevents WHERE labevents.itemid IN ( SELECT d_labitems.itemid FROM d_labitems WHERE d_labitems.label = 'red blood cells' ) AND labevents.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.age = ( SELECT admissions.age FROM admissions WHERE admissions.subject_id = 10026406 AND admissions.dischtime IS NOT NULL ORDER BY admissions.admittime ASC LIMIT 1 ) ) ) AS T1 WHERE T1.valuenum = 4.46"
2,b0b4926abc6fba2f9b78a017,"SELECT COUNT( DISTINCT T1.subject_id ) FROM ( SELECT admissions.subject_id, diagnoses_icd.charttime, admissions.hadm_id FROM diagnoses_icd JOIN admissions ON diagnoses_icd.hadm_id = admissions.hadm_id WHERE diagnoses_icd.icd_code = ( SELECT d_icd_diagnoses.icd_code FROM d_icd_diagnoses WHERE d_icd_diagnoses.long_title = 'presence of cardiac pacemaker' ) AND datetime(diagnoses_icd.charttime,'start of year') = datetime('2100-12-31 23:59:00','start of year','-0 year') ) AS T1 JOIN ( SELECT admissions.subject_id, diagnoses_icd.charttime, admissions.hadm_id FROM diagnoses_icd JOIN admissions ON diagnoses_icd.hadm_id = admissions.hadm_id WHERE diagnoses_icd.icd_code = ( SELECT d_icd_diagnoses.icd_code FROM d_icd_diagnoses WHERE d_icd_diagnoses.long_title = 'hyperlipidemia, unspecified' ) AND datetime(diagnoses_icd.charttime,'start of year') = datetime('2100-12-31 23:59:00','start of year','-0 year') ) AS T2 ON T1.subject_id = T2.subject_id WHERE T1.charttime < T2.charttime AND T1.hadm_id = T2.hadm_id","SELECT COUNT( DISTINCT T1.subject_id ) FROM ( SELECT admissions.subject_id, diagnoses_icd.charttime, admissions.hadm_id FROM diagnoses_icd JOIN admissions ON diagnoses_icd.hadm_id = admissions.hadm_id WHERE diagnoses_icd.icd_code = ( SELECT d_icd_diagnoses.icd_code FROM d_icd_diagnoses WHERE d_icd_diagnoses.long_title = 'presence of cardiac pacemaker' ) AND datetime(diagnoses_icd.charttime,'start of year') = datetime('2100-12-31 23:59:00','start of year','-0 year') ) AS T1 JOIN ( SELECT admissions.subject_id, diagnoses_icd.charttime, admissions.hadm_id FROM diagnoses_icd JOIN admissions ON diagnoses_icd.hadm_id = admissions.hadm_id WHERE diagnoses_icd.icd_code = ( SELECT d_icd_diagnoses.icd_code FROM d_icd_diagnoses WHERE d_icd_diagnoses.long_title = 'hyperlipidemia, unspecified' ) AND datetime(diagnoses_icd.charttime,'start of year') = datetime('2100-12-31 23:59:00','start of year','-0 year') ) AS T2 ON T1.subject_id = T2.su

In [41]:
import os
from scoring_program.scoring_utils import execute_all, reliability_score, penalize
DB_PATH = os.path.join('data', 'mimic_iv', 'mimic_iv.sqlite')

pred_res = execute_all(dict, db_path=DB_PATH, tag='pred')
label_res = execute_all(label, db_path=DB_PATH, tag='real')

In [42]:
# add results according to the id
df["pred_res"] = [pred_res[id] for id in df["id"]]
df["label_res"] = [label_res[id] for id in df["id"]]
df.head(10)

,id,pred,label,pred_res,label_res
0,3b9849548e56c59f768d5447,SELECT MIN(chartevents.valuenum) FROM chartevents WHERE chartevents.stay_id IN ( SELECT icustays.stay_id FROM icustays WHERE icustays.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10021118 ) AND icustays.outtime IS NOT NULL ORDER BY icustays.intime ASC LIMIT 1 ) AND chartevents.itemid IN ( SELECT d_items.itemid FROM d_items WHERE d_items.label = 'respiratory rate' AND d_items.linksto = 'chartevents' ),SELECT MIN(chartevents.valuenum) FROM chartevents WHERE chartevents.stay_id IN ( SELECT icustays.stay_id FROM icustays WHERE icustays.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10021118 ) AND icustays.outtime IS NOT NULL ORDER BY icustays.intime ASC LIMIT 1 ) AND chartevents.itemid IN ( SELECT d_items.itemid FROM d_items WHERE d_items.label = 'respiratory rate' AND d_items.linksto = 'chartevents' ),[['5.0']],[['5.0']]
1,3f720ea93f94700efe7fe7ee,"SELECT DISTINCT T1.C1 FROM ( SELECT labevents.valuenum, PERCENT_RANK() OVER ( ORDER BY labevents.valuenum ) AS C1 FROM labevents WHERE labevents.itemid IN ( SELECT d_labitems.itemid FROM d_labitems WHERE d_labitems.label = 'red blood cells' ) AND labevents.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.age = ( SELECT admissions.age FROM admissions WHERE admissions.subject_id = 10026406 AND admissions.dischtime IS NOT NULL ORDER BY admissions.admittime ASC LIMIT 1 ) ) ) AS T1 WHERE T1.valuenum = 4.46","SELECT DISTINCT T1.C1 FROM ( SELECT labevents.valuenum, PERCENT_RANK() OVER ( ORDER BY labevents.valuenum ) AS C1 FROM labevents WHERE labevents.itemid IN ( SELECT d_labitems.itemid FROM d_labitems WHERE d_labitems.label = 'red blood cells' ) AND labevents.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.age = ( SELECT admissions.age FROM admissions WHERE admissions.subject_id = 10026406 AND admissions.dischtime IS NOT NULL ORDER BY admissions.admittime ASC LIMIT 1 ) ) ) AS T1 WHERE T1.valuenum = 4.46",[['0.933']],[['0.933']]
2,b0b4926abc6fba2f9b78a017,"SELECT COUNT( DISTINCT T1.subject_id ) FROM ( SELECT admissions.subject_id, diagnoses_icd.charttime, admissions.hadm_id FROM diagnoses_icd JOIN admissions ON diagnoses_icd.hadm_id = admissions.hadm_id WHERE diagnoses_icd.icd_code = ( SELECT d_icd_diagnoses.icd_code FROM d_icd_diagnoses WHERE d_icd_diagnoses.long_title = 'presence of cardiac pacemaker' ) AND datetime(diagnoses_icd.charttime,'start of year') = datetime('2100-12-31 23:59:00','start of year','-0 year') ) AS T1 JOIN ( SELECT admissions.subject_id, diagnoses_icd.charttime, admissions.hadm_id FROM diagnoses_icd JOIN admissions ON diagnoses_icd.hadm_id = admissions.hadm_id WHERE diagnoses_icd.icd_code = ( SELECT d_icd_diagnoses.icd_code FROM d_icd_diagnoses WHERE d_icd_diagnoses.long_title = 'hyperlipidemia, unspecified' ) AND datetime(diagnoses_icd.charttime,'start of year') = datetime('2100-12-31 23:59:00','start of year','-0 year') ) AS T2 ON T1.subject_id = T2.subject_id WHERE T1.charttime < T2.charttime AND T1.hadm_id = T2.hadm_id","SELECT COUNT( DISTINCT T1.subject_id ) FROM ( SELECT admissions.subject_id, diagnoses_icd.charttime, admissions.hadm_id FROM diagnoses_icd JOIN admissions ON diagnoses_icd.hadm_id = admissions.hadm_id WHERE diagnoses_icd.icd_code = ( SELECT d_icd_diagnoses.icd_code FROM d_icd_diagnoses WHERE d_icd_diagnoses.long_title = 'presence of cardiac pacemaker' ) AND datetime(diagnoses_icd.charttime,'start of year') = datetime('2100-12-31 23:59:00','start of year','-0 year') ) AS T1 JOIN ( SELECT admissions.subject_id, diagnoses_icd.charttime, admissions.hadm_id FROM diagnoses_icd JOIN admissions ON diagnoses_icd.hadm_id = admissions.hadm_id WHERE diagnoses_icd.icd_code = ( SELECT d_icd_diagnoses.icd_code FROM d_icd_diagnoses WHERE d_icd_diagnoses.long_title = 'hyperlipidemia, unspecified' ) AND datetime(diagnoses_icd.charttime,'start of year') = datetime('2100-12-31 23:59:

In [44]:
# get rows where pred != label
diff_rows = df[(df["pred"] != df["label"]) & (df["pred_res"] != df["label_res"])]
print(f"Number of different rows: {len(diff_rows)}")

diff_rows.to_csv("./train_diff_rows.csv", index=False) # when changing =' to = ', the result changes from 120 to 43

Number of different rows: 43


In [45]:
# set maximum display for values
pd.set_option('display.max_colwidth', None)
# reset index for diff_rows
diff_rows = diff_rows.reset_index(drop=True)
diff_rows.head()

,id,pred,label,pred_res,label_res
0,1f02c929b05a2d1eae19aa43,"SELECT chartevents.charttime FROM chartevents WHERE chartevents.stay_id IN ( SELECT icustays.stay_id FROM icustays WHERE icustays.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10021666 ) ) AND chartevents.itemid IN ( SELECT d_items.itemid FROM d_items WHERE d_items.label = 'heart rate' AND d_items.linksto = 'chartevents' ) AND strftime('%Y-%m-%d',chartevents.charttime) = '2100-03-21' ORDER BY chartevents.valuenum ASC, chartevents.charttime DESC LIMIT 1","SELECT chartevents.charttime FROM chartevents WHERE chartevents.stay_id IN ( SELECT icustays.stay_id FROM icustays WHERE icustays.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10021666 ) ) AND chartevents.itemid IN ( SELECT d_items.itemid FROM d_items WHERE d_items.label = 'heart rate' AND d_items.linksto = 'chartevents' ) AND strftime('%Y-%m-%d',chartevents.charttime) = '2100-03-31' ORDER BY chartevents.valuenum ASC, chartevents.charttime DESC LIMIT 1",[],[['2100-03-31 16:00:00']]
1,adb4dc30adc6acdfe49dc9c0,SELECT ( SELECT chartevents.valuenum FROM chartevents WHERE chartevents.stay_id IN ( SELECT icustays.stay_id FROM icustays WHERE icustays.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10005817 ) AND icustays.outtime IS NOT NULL ORDER BY icustays.intime ASC LIMIT 1 ) AND chartevents.itemid IN ( SELECT d_items.itemid FROM d_items WHERE d_items.label = 'arterial blood pressure mean' AND d_items.linksto = 'chartevents' ) ORDER BY chartevents.charttime DESC LIMIT 1 ) - ( SELECT chartevents.valuenum FROM chartevents WHERE chartevents.stay_id IN ( SELECT icustays.stay_id FROM icustays WHERE icustays.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10005817 ) AND icustays.outtime IS NOT NULL ORDER BY icustays.intime ASC LIMIT 1 ) AND chartevents.itemid IN ( SELECT d_items.itemid FROM d_items WHERE d_items.label = 'arterial blood pressure mean' AND d_items.linksto = 'chartevents' ) ORDER BY chartevents.charttime DESC LIMIT 1 OFFSET 1 ),SELECT ( SELECT chartevents.valuenum FROM chartevents WHERE chartevents.stay_id IN ( SELECT icustays.stay_id FROM icustays WHERE icustays.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10005817 ) AND icustays.outtime IS NOT NULL ORDER BY icustays.intime ASC LIMIT 1 ) AND chartevents.itemid IN ( SELECT d_items.itemid FROM d_items WHERE d_items.label = 'arterial blood pressure mean' AND d_items.linksto = 'chartevents' ) ORDER BY chartevents.charttime DESC LIMIT 1 ) < ( SELECT chartevents.valuenum FROM chartevents WHERE chartevents.stay_id IN ( SELECT icustays.stay_id FROM icustays WHERE icustays.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10005817 ) AND icustays.outtime IS NOT NULL ORDER BY icustays.intime ASC LIMIT 1 ) AND chartevents.itemid IN ( SELECT d_items.itemid FROM d_items WHERE d_items.label = 'arterial blood pressure mean' AND d_items.linksto = 'chartevents' ) ORDER BY chartevents.charttime DESC LIMIT 1 OFFSET 1 ),[['7.0']],[['0.0']]
2,c4583562855d6e407a1c88ed,"SELECT MIN(T1.C1) FROM ( SELECT COUNT( DISTINCT diagnoses_icd.hadm_id ) AS C1 FROM diagnoses_icd WHERE diagnoses_icd.icd_code = ( SELECT d_icd_diagnoses.icd_code FROM d_icd_diagnoses WHERE d_icd_diagnoses.long_title = 'family history of falling' ) GROUP BY strftime('%Y',diagnoses_icd.charttime) ) AS T1","SELECT MIN(T1.C1) FROM ( SELECT COUNT( DISTINCT diagnoses_icd.hadm_id ) AS C1 FROM diagnoses_icd WHERE diagnoses_icd.icd_code = ( SELECT d_icd_diagnoses.icd_code FROM d_icd_diagnoses WHERE d_icd_diagnoses.long_title = 'history of falling' ) GROUP BY strftime('%Y',diagnoses_icd.charttime) ) AS T1",[['None']],[['2.0']]
3,06f4ed18e0175feb847ceb18,"SELECT COUNT( DISTINCT admissions.subject_id ) FROM admissions WHERE admissions.hadm_id IN ( SELECT prescriptions.hadm_id FROM prescriptions WHERE prescri

In [46]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [47]:
diff_rows["editdistance"] = diff_rows.apply(lambda x: levenshteinDistance(x.pred, x.label), axis=1)
diff_rows.head()

,id,pred,label,pred_res,label_res,editdistance
0,1f02c929b05a2d1eae19aa43,"SELECT chartevents.charttime FROM chartevents WHERE chartevents.stay_id IN ( SELECT icustays.stay_id FROM icustays WHERE icustays.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10021666 ) ) AND chartevents.itemid IN ( SELECT d_items.itemid FROM d_items WHERE d_items.label = 'heart rate' AND d_items.linksto = 'chartevents' ) AND strftime('%Y-%m-%d',chartevents.charttime) = '2100-03-21' ORDER BY chartevents.valuenum ASC, chartevents.charttime DESC LIMIT 1","SELECT chartevents.charttime FROM chartevents WHERE chartevents.stay_id IN ( SELECT icustays.stay_id FROM icustays WHERE icustays.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10021666 ) ) AND chartevents.itemid IN ( SELECT d_items.itemid FROM d_items WHERE d_items.label = 'heart rate' AND d_items.linksto = 'chartevents' ) AND strftime('%Y-%m-%d',chartevents.charttime) = '2100-03-31' ORDER BY chartevents.valuenum ASC, chartevents.charttime DESC LIMIT 1",[],[['2100-03-31 16:00:00']],1
1,adb4dc30adc6acdfe49dc9c0,SELECT ( SELECT chartevents.valuenum FROM chartevents WHERE chartevents.stay_id IN ( SELECT icustays.stay_id FROM icustays WHERE icustays.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10005817 ) AND icustays.outtime IS NOT NULL ORDER BY icustays.intime ASC LIMIT 1 ) AND chartevents.itemid IN ( SELECT d_items.itemid FROM d_items WHERE d_items.label = 'arterial blood pressure mean' AND d_items.linksto = 'chartevents' ) ORDER BY chartevents.charttime DESC LIMIT 1 ) - ( SELECT chartevents.valuenum FROM chartevents WHERE chartevents.stay_id IN ( SELECT icustays.stay_id FROM icustays WHERE icustays.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10005817 ) AND icustays.outtime IS NOT NULL ORDER BY icustays.intime ASC LIMIT 1 ) AND chartevents.itemid IN ( SELECT d_items.itemid FROM d_items WHERE d_items.label = 'arterial blood pressure mean' AND d_items.linksto = 'chartevents' ) ORDER BY chartevents.charttime DESC LIMIT 1 OFFSET 1 ),SELECT ( SELECT chartevents.valuenum FROM chartevents WHERE chartevents.stay_id IN ( SELECT icustays.stay_id FROM icustays WHERE icustays.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10005817 ) AND icustays.outtime IS NOT NULL ORDER BY icustays.intime ASC LIMIT 1 ) AND chartevents.itemid IN ( SELECT d_items.itemid FROM d_items WHERE d_items.label = 'arterial blood pressure mean' AND d_items.linksto = 'chartevents' ) ORDER BY chartevents.charttime DESC LIMIT 1 ) < ( SELECT chartevents.valuenum FROM chartevents WHERE chartevents.stay_id IN ( SELECT icustays.stay_id FROM icustays WHERE icustays.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10005817 ) AND icustays.outtime IS NOT NULL ORDER BY icustays.intime ASC LIMIT 1 ) AND chartevents.itemid IN ( SELECT d_items.itemid FROM d_items WHERE d_items.label = 'arterial blood pressure mean' AND d_items.linksto = 'chartevents' ) ORDER BY chartevents.charttime DESC LIMIT 1 OFFSET 1 ),[['7.0']],[['0.0']],1
2,c4583562855d6e407a1c88ed,"SELECT MIN(T1.C1) FROM ( SELECT COUNT( DISTINCT diagnoses_icd.hadm_id ) AS C1 FROM diagnoses_icd WHERE diagnoses_icd.icd_code = ( SELECT d_icd_diagnoses.icd_code FROM d_icd_diagnoses WHERE d_icd_diagnoses.long_title = 'family history of falling' ) GROUP BY strftime('%Y',diagnoses_icd.charttime) ) AS T1","SELECT MIN(T1.C1) FROM ( SELECT COUNT( DISTINCT diagnoses_icd.hadm_id ) AS C1 FROM diagnoses_icd WHERE diagnoses_icd.icd_code = ( SELECT d_icd_diagnoses.icd_code FROM d_icd_diagnoses WHERE d_icd_diagnoses.long_title = 'history of falling' ) GROUP BY strftime('%Y',diagnoses_icd.charttime) ) AS T1",[['None']],[['2.0']],7
3,06f4ed18e0175feb847ceb18,"SELECT COUNT( DISTINCT admissions.subject_id ) FROM admissions WHERE admissions.hadm_id IN ( SELECT prescriptions.hadm_id FROM prescrip

In [49]:
diff_rows.tail()

,id,pred,label,pred_res,label_res,editdistance
38,e530d92c14e1178895ca7c89,"SELECT T1.starttime FROM ( SELECT admissions.subject_id, prescriptions.starttime FROM prescriptions JOIN admissions ON prescriptions.hadm_id = admissions.hadm_id WHERE prescriptions.drug = 'heparin flush (10 units/ml)' AND admissions.subject_id = 10020740 AND datetime(prescriptions.starttime,'start of year') = datetime('2100-12-31 23:59:00','start of year','-0 year') AND strftime('%m',prescriptions.starttime) = '04' ) AS T1 JOIN ( SELECT admissions.subject_id, prescriptions.starttime FROM prescriptions JOIN admissions ON prescriptions.hadm_id = admissions.hadm_id WHERE prescriptions.drug = 'lidocaine 1%' AND admissions.subject_id = 10020740 AND datetime(prescriptions.starttime,'start of year') = datetime('2100-12-31 23:59:00','start of year','-0 year') AND strftime('%m',prescriptions.starttime) = '04' ) AS T2 ON T1.subject_id = T2.subject_id WHERE datetime(T1.starttime) = datetime(T2.starttime) ORDER BY T1.starttime DESC LIMIT 1","SELECT T1.starttime FROM ( SELECT admissions.subject_id, prescriptions.starttime FROM prescriptions JOIN admissions ON prescriptions.hadm_id = admissions.hadm_id WHERE prescriptions.drug = 'lidocaine 1%' AND admissions.subject_id = 10020740 AND datetime(prescriptions.starttime,'start of year') = datetime('2100-12-31 23:59:00','start of year','-0 year') AND strftime('%m',prescriptions.starttime) = '04' ) AS T1 JOIN ( SELECT admissions.subject_id, prescriptions.starttime FROM prescriptions JOIN admissions ON prescriptions.hadm_id = admissions.hadm_id WHERE prescriptions.drug = 'heparin flush (10 units/ml)' AND admissions.subject_id = 10020740 AND datetime(prescriptions.starttime,'start of year') = datetime('2100-12-31 23:59:00','start of year','-0 year') AND strftime('%m',prescriptions.starttime) = '04' ) AS T2 ON T1.subject_id = T2.subject_id WHERE datetime(T1.starttime) = datetime(T2.starttime) ORDER BY T1.starttime DESC LIMIT 1",[['2100-04-22 10:00:00']],[],48
39,c0a87bdad4a3ac34d3210642,SELECT labevents.valuenum FROM labevents WHERE labevents.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 100220431 ) AND labevents.itemid IN ( SELECT d_labitems.itemid FROM d_labitems WHERE d_labitems.label = 'vitamin b12' ) ORDER BY labevents.charttime ASC LIMIT 1,SELECT labevents.valuenum FROM labevents WHERE labevents.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10022041 ) AND labevents.itemid IN ( SELECT d_labitems.itemid FROM d_labitems WHERE d_labitems.label = 'vitamin b12' ) ORDER BY labevents.charttime ASC LIMIT 1,[],[['971.0']],1
40,771257ca5b02630533cafe73,"SELECT COUNT( DISTINCT admissions.subject_id ) FROM admissions WHERE admissions.dischtime is NOT NULL AND strftime('%Y',admissions.dischtime) >= '2100'","SELECT COUNT( DISTINCT admissions.subject_id ) FROM admissions WHERE admissions.hadm_id IN ( SELECT transfers.hadm_id FROM transfers WHERE transfers.careunit = 'discharge lounge' AND strftime('%Y',transfers.intime) >= '2100' )",[['90.0']],[['19.0']],102
41,dcc6094fae5bc1bb4de3c989,"SELECT labevents.charttime FROM labevents WHERE labevents.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10004733 AND admissions.dischtime IS NULL ) AND labevents.itemid IN ( SELECT d_labitems.itemid FROM d_labitems WHERE d_labitems.label = 'mini protein' ) ORDER BY labevents.valuenum ASC, labevents.charttime DESC LIMIT 1","SELECT labevents.charttime FROM labevents WHERE labevents.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10004733 AND admissions.dischtime IS NULL ) AND labevents.itemid IN ( SELECT d_labitems.itemid FROM d_labitems WHERE d_labitems.label = 'protein' ) ORDER BY labevents.valuenum ASC, labevents.charttime DESC LIMIT 1",[],[['2100-12-31 11:03:00']],5
42,adf57bf6800cbd7a1bcce75f,"SELECT labevents.charttime FROM labevents WHERE labevents.hadm_id IN ( SELECT admissions.hadm_id FROM adm

In [50]:
# get statistics for diff_rows["editdistance"]
from matplotlib import pyplot as plt

diff_rows["editdistance"].describe()

count      43.000000
mean       59.372093
std       181.716030
min         1.000000
25%         2.000000
50%         7.000000
75%        48.000000
max      1177.000000
Name: editdistance, dtype: float64

In [51]:
big_diff_rows = diff_rows[diff_rows["editdistance"] > 3]
print(f"Number of big difference rows: {len(big_diff_rows)}")
big_diff_rows.head()

Number of big difference rows: 30


,id,pred,label,pred_res,label_res,editdistance
2,c4583562855d6e407a1c88ed,"SELECT MIN(T1.C1) FROM ( SELECT COUNT( DISTINCT diagnoses_icd.hadm_id ) AS C1 FROM diagnoses_icd WHERE diagnoses_icd.icd_code = ( SELECT d_icd_diagnoses.icd_code FROM d_icd_diagnoses WHERE d_icd_diagnoses.long_title = 'family history of falling' ) GROUP BY strftime('%Y',diagnoses_icd.charttime) ) AS T1","SELECT MIN(T1.C1) FROM ( SELECT COUNT( DISTINCT diagnoses_icd.hadm_id ) AS C1 FROM diagnoses_icd WHERE diagnoses_icd.icd_code = ( SELECT d_icd_diagnoses.icd_code FROM d_icd_diagnoses WHERE d_icd_diagnoses.long_title = 'history of falling' ) GROUP BY strftime('%Y',diagnoses_icd.charttime) ) AS T1",[['None']],[['2.0']],7
3,06f4ed18e0175feb847ceb18,"SELECT COUNT( DISTINCT admissions.subject_id ) FROM admissions WHERE admissions.hadm_id IN ( SELECT prescriptions.hadm_id FROM prescriptions WHERE prescriptions.drug = 'alka-magnesium hydrox.-simethicone' AND strftime('%Y',prescriptions.starttime) = '2100' )","SELECT COUNT( DISTINCT admissions.subject_id ) FROM admissions WHERE admissions.hadm_id IN ( SELECT prescriptions.hadm_id FROM prescriptions WHERE prescriptions.drug = 'aluminum-magnesium hydrox.-simethicone' AND strftime('%Y',prescriptions.starttime) = '2100' )",[['0.0']],[['4.0']],6
4,32b1864cd437a05f94495679,"SELECT COUNT( DISTINCT admissions.subject_id ) FROM admissions WHERE admissions.hadm_id IN ( SELECT prescriptions.hadm_id FROM prescriptions WHERE prescriptions.drug = 'medicine' AND strftime('%Y',prescriptions.starttime) >= '2100' )","SELECT COUNT( DISTINCT admissions.subject_id ) FROM admissions WHERE admissions.hadm_id IN ( SELECT transfers.hadm_id FROM transfers WHERE transfers.careunit = 'medicine' AND strftime('%Y',transfers.intime) >= '2100' )",[['0.0']],[['19.0']],55
6,e0d157e7655d68add3bb76d5,"SELECT DISTINCT cost.cost FROM cost WHERE cost.event_type = 'diagnoses_icd' AND cost.event_id IN ( SELECT diagnoses_icd.row_id FROM diagnoses_icd WHERE diagnoses_icd.icd_code = ( SELECT d_icd_diagnoses.icd_code FROM d_icd_diagnoses WHERE d_icd_diagnoses.long_title = 'astomosis, bypass, or graft, with natural or artificial tissues used as implant causing abnormal reaction, or later complication, without mention of misadventure at time of operation' ) )","SELECT DISTINCT cost.cost FROM cost WHERE cost.event_type = 'diagnoses_icd' AND cost.event_id IN ( SELECT diagnoses_icd.row_id FROM diagnoses_icd WHERE diagnoses_icd.icd_code = ( SELECT d_icd_diagnoses.icd_code FROM d_icd_diagnoses WHERE d_icd_diagnoses.long_title = 'surgical operation with anastomosis, bypass, or graft, with natural or artificial tissues used as implant causing abnormal patient reaction, or later complication, without mention of misadventure at time of operation' ) )",[],[['6.95']],34
8,aea39d595c762f58e94e6bc6,"SELECT labevents.valuenum FROM labevents WHERE labevents.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10021312 ) AND labevents.itemid IN ( SELECT d_labitems.itemid FROM d_labitems WHERE d_labitems.label = 'measured absolute eosinophil count' ) AND strftime('%Y-%m',labevents.charttime) >= '2100-01' ORDER BY labevents.charttime ASC LIMIT 1","SELECT labevents.valuenum FROM labevents WHERE labevents.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10021312 ) AND labevents.itemid IN ( SELECT d_labitems.itemid FROM d_labitems WHERE d_labitems.label = 'absolute eosinophil count' ) AND strftime('%Y-%m',labevents.charttime) >= '2100-01' ORDER BY labevents.charttime ASC LIMIT 1",[],[['0.0']],9


In [52]:
idx = 0
# iterrate over diff_rows.loc[idx].pred, diff_rows.loc[idx].label
print(f"ID: {diff_rows.loc[idx].id}")
print(f"Pred : {diff_rows.loc[idx].pred}")
print(f"Label: {diff_rows.loc[idx].label}")
for word in diff_rows.loc[idx].pred.split():
  if word in diff_rows.loc[idx].label.split():
    print(f"Same word: {word}")
  else:
    print(f"Different word: {word}")

ID: 1f02c929b05a2d1eae19aa43
Pred : SELECT chartevents.charttime FROM chartevents WHERE chartevents.stay_id IN ( SELECT icustays.stay_id FROM icustays WHERE icustays.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10021666 ) ) AND chartevents.itemid IN ( SELECT d_items.itemid FROM d_items WHERE d_items.label = 'heart rate' AND d_items.linksto = 'chartevents' ) AND strftime('%Y-%m-%d',chartevents.charttime) = '2100-03-21' ORDER BY chartevents.valuenum ASC, chartevents.charttime DESC LIMIT 1
Label: SELECT chartevents.charttime FROM chartevents WHERE chartevents.stay_id IN ( SELECT icustays.stay_id FROM icustays WHERE icustays.hadm_id IN ( SELECT admissions.hadm_id FROM admissions WHERE admissions.subject_id = 10021666 ) ) AND chartevents.itemid IN ( SELECT d_items.itemid FROM d_items WHERE d_items.label = 'heart rate' AND d_items.linksto = 'chartevents' ) AND strftime('%Y-%m-%d',chartevents.charttime) = '2100-03-31' ORDER BY chartevents.valuenum ASC, 